# Calculate the rs-fMRI functional connectivity (FC) matrix for each HCP subject 
## Schaefer 400, 7 network parcellation

This is done for each scan session (day1 LR, day1 RL, day2 LR, day2 RL) by subject, then calculates the mean FC matrix from the 4 session matrices for each subject and saves it

## Load packages

In [2]:
# General
import numpy as np
from numpy import inf
import pandas as pd
import os

# Computing
import scipy.io
import sklearn 

# Neuroimaging
import nibabel as nib
from nilearn.datasets import fetch_atlas_schaefer_2018
import hcp_utils as hcp

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


## Define directories

In [3]:
codedir = os.path.abspath('')  # obtain current direction from which script is runnning

datadir_day2 = '/data/p_02378/data/hcp_rsfunc/hcp-functional-connectivity/'

datadir_day1 = '/data/p_02378/data/hcp_rsfunc/HCP_S1200_rfMRI_DAY1/'

dataout = '/data/p_02667/sex_diff_gradients/data/fc_matrices/schaefer_output/'

In [4]:
path_list = os.listdir(datadir_day2)
path_list.sort()

## Loading Schaefer 400 atlas (7 network) and adapting it to HCP data

In [5]:
# Schaefer_400 - 7 networks
dlabel_400 = nib.load('/data/p_02667/sex_diff_gradients/data/schaefer400/Schaefer2018_400Parcels_7Networks_order.dlabel.nii').get_fdata()

# fs_LR_32k is a standard brain surface (like MNI space) -> LR means the first 32k belong to left hemisphere, the second belong to right hemisphere
mat = scipy.io.loadmat('/data/p_02667/sex_diff_gradients/data/schaefer400/fs_LR_32k_medial_mask.mat')


# HCP cortex data does not contain medial_wall using python, we need to delete it in Schaefer template
mask = mat['medial_mask']
schaefer_400 = dlabel_400.T[np.where(mask==1)]


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


Bin´s instructions
- find the schaefer400 dlabel nii file (which is 64k (vertices?), cortex also 64k*1200 time series
- find cortex[schaefer==i] for i in range(400), then average
- then you get (400, 1200) ts
- then do np.corrcoef(ts) to get the correlation matrix
- But of note, some file contains medial wall (64k points) but some file excludes medial wall (59k points)

## Scan session day 2 - LR

In [5]:
# read the time series of scan session day 2, LR, ** rfMRI_REST2_LR **
# parcellate the time series to Glasser 360 (hcp.parcellate(data, hcp.mmp)) and then adapte it to Schaefer
# compute the functional connectivity matrix and save

for dir in path_list:
    if os.path.exists(datadir_day2+dir+'/MNINonLinear/Results/rfMRI_REST2_LR/rfMRI_REST2_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii'):
        print('executing subject: '+dir+'......')
        
        img = nib.load(datadir_day2+dir+'/MNINonLinear/Results/rfMRI_REST2_LR/rfMRI_REST2_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii')
        data = img.get_fdata()
        data_parc = hcp.parcellate(data, hcp.mmp)
        data_sub = data_parc[:,360:]  # this is the subcortical regions parcels: include if you want 419x419 matrices
        
        img_cortex = data[:,hcp.struct.cortex] # cortex vertices
        node = [None]*400 # data_sub.shape[0] is time points. not all data has 1200 ts
        
        for i in range(400):
            a = img_cortex[:,np.where(schaefer_400==i+1)][:,0]
            ts = [None] * data_sub.shape[0]
        
            for n in range(data_sub.shape[0]):
                ts[n] = a[n].mean()
                node[i] = ts
      
        # cortex parcels finished, now concatenate cortex and subcortex
        data_cor = np.array(node)
        
        # if you want matrices to include cortical and subcortical regions parcels, concatenate them here (so that in 419x419 matrices, the last 19 are subcortical parcels)
        #data_cor_sub = np.concatenate((data_cor,data_sub.T))  
        
        # correlation
        corr_matrix = np.corrcoef(data_cor)   # use data_cor_sub if I want to include subcortical areas to yield 419x419 matrices
        corr_matrix[corr_matrix>0.99999]=1  # if 0.99999 make it 1 - for some unknown reason some correlations of same parcels are not round 1 so forcing it into 1
        
        # save
        fname_corr_matrix = dataout + 'DAY2_LR/corr/'+ dir +'.csv'
        np.savetxt(fname_corr_matrix, corr_matrix, delimiter = ',')
        print('finished subject: '+dir+'......')
    
    else:
        fname_corr_matrix_failed = dataout + 'DAY2_LR/file_failed.txt'
        fail_file = open(fname_corr_matrix_failed, 'a')
        fail_file.write(dir+'\n')
        fail_file.close()
        print('subject '+dir+' file dose not exist.....')
        

subject .datalad file dose not exist.....
subject .git file dose not exist.....
subject .gitattributes file dose not exist.....
subject .gitmodules file dose not exist.....
subject .hcp file dose not exist.....
executing subject: 100206......


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


finished subject: 100206......
executing subject: 100307......


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


finished subject: 100307......
executing subject: 100408......


...

finished subject: 993675......
executing subject: 994273......


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


finished subject: 994273......
subject 995174 file dose not exist.....
executing subject: 996782......


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


finished subject: 996782......
subject DATA_USE_AGREEMENT.md file dose not exist.....
subject README.md file dose not exist.....


## Scan session day 2 - RL

In [6]:
# read the time series of scan session day 2, RL, ** rfMRI_REST2_RL **
# parcellate the time series to Glasser 360 (hcp.parcellate(data, hcp.mmp)) and then adapte it to Schaefer
# compute the functional connectivity matrix and save
        
for dir in path_list:
    if os.path.exists(datadir_day2+dir+'/MNINonLinear/Results/rfMRI_REST2_RL/rfMRI_REST2_RL_Atlas_MSMAll_hp2000_clean.dtseries.nii'):
        print('executing subject: '+dir+'......')
        
        img = nib.load(datadir_day2+dir+'/MNINonLinear/Results/rfMRI_REST2_RL/rfMRI_REST2_RL_Atlas_MSMAll_hp2000_clean.dtseries.nii')
        data = img.get_fdata()
        data_parc = hcp.parcellate(data, hcp.mmp)
        data_sub = data_parc[:,360:]  # this is the subcortical regions parcels: include if you want 419x419 matrices
        
        img_cortex = data[:,hcp.struct.cortex] # cortex vertices
        node = [None]*400 # data_sub.shape[0] is time points. not all data has 1200 ts
        
        for i in range(400):
            a = img_cortex[:,np.where(schaefer_400==i+1)][:,0]
            ts = [None] * data_sub.shape[0]
        
            for n in range(data_sub.shape[0]):
                ts[n] = a[n].mean()
                node[i] = ts
      
        # cortex parcels finished, now concatenate cortex and subcortex
        data_cor = np.array(node)

        # if you want matrices to include cortical and subcortical regions parcels, concatenate them here (so that in 419x419 matrices, the last 19 are subcortical parcels)
        #data_cor_sub = np.concatenate((data_cor,data_sub.T))  
        
        # correlation
        corr_matrix = np.corrcoef(data_cor)  # use data_cor_sub if I want to include subcortical areas to yield 419x419 matrices
        corr_matrix[corr_matrix>0.99999]=1  # if 0.99999 make it 1 - for some unknown reason some correlations of same parcels are not round 1 so forcing it into 1
        
        # save
        fname_corr_matrix = dataout + 'DAY2_RL/corr/'+ dir +'.csv'
        np.savetxt(fname_corr_matrix, corr_matrix, delimiter = ',')
        print('finished subject: '+dir+'......')
    
    else:
        fname_corr_matrix_failed = dataout + 'DAY2_RL/file_failed.txt'
        fail_file = open(fname_corr_matrix_failed, 'a')
        fail_file.write(dir+'\n')
        fail_file.close()
        print('subject '+dir+' file dose not exist.....')


executing subject: 100206......


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


finished subject: 100206......


## Scan session day 1 - LR

In [7]:
# read the time series of scan session day 1, LR, ** rfMRI_REST1_LR **,
# parcellate the time series to Glasser 360 (hcp.parcellate(data, hcp.mmp)) and then adapte it to Schaefer
# compute the functional connectivity matrix and save

for dir in path_list:
    if os.path.exists(datadir_day1+dir+'/MNINonLinear/Results/rfMRI_REST1_LR/rfMRI_REST1_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii'):
        print('executing subject: '+dir+'......')
        
        img = nib.load(datadir_day1+dir+'/MNINonLinear/Results/rfMRI_REST1_LR/rfMRI_REST1_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii')
        data = img.get_fdata()
        data_parc = hcp.parcellate(data, hcp.mmp)
        data_sub = data_parc[:,360:]  # this is the subcortical regions parcels: include if you want 419x419 matrices
        
        img_cortex = data[:,hcp.struct.cortex] # cortex vertices
        node = [None]*400 # data_sub.shape[0] is time points. not all data has 1200 ts
        
        for i in range(400):
            a = img_cortex[:,np.where(schaefer_400==i+1)][:,0]
            ts = [None] * data_sub.shape[0]
        
            for n in range(data_sub.shape[0]):
                ts[n] = a[n].mean()
                node[i] = ts
      
        # cortex parcels finished, now concatenate cortex and subcortex
        data_cor = np.array(node)
        
        # if you want matrices to include cortical and subcortical regions parcels, concatenate them here (so that in 419x419 matrices, the last 19 are subcortical parcels)
        #data_cor_sub = np.concatenate((data_cor,data_sub.T))  
        
        # correlation
        corr_matrix = np.corrcoef(data_cor)  # use data_cor_sub if I want to include subcortical areas to yield 419x419 matrices
        corr_matrix[corr_matrix>0.99999]=1  # if 0.99999 make it 1 - for some unknown reason some correlations of same parcels are not round 1 so forcing it into 1
        
        # save
        fname_corr_matrix = dataout + 'DAY1_LR/corr/'+ dir +'.csv'
        np.savetxt(fname_corr_matrix, corr_matrix, delimiter = ',')
        print('finished subject: '+dir+'......')
    
    else:
        fname_corr_matrix_failed = dataout + 'DAY1_LR/file_failed.txt'
        fail_file = open(fname_corr_matrix_failed, 'a')
        fail_file.write(dir+'\n')
        fail_file.close()
        print('subject '+dir+' file dose not exist.....')


executing subject: 100206......


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


finished subject: 100206......


## Scan session day 1 - RL

In [8]:
# read the time series of scan session day 1, RL, ** rfMRI_REST1_RL **,
# parcellate the time series to Glasser 360 (hcp.parcellate(data, hcp.mmp)) and then adapte it to Schaefer
# compute the functional connectivity matrix and save

for dir in path_list:
    if os.path.exists(datadir_day1+dir+'/MNINonLinear/Results/rfMRI_REST1_RL/rfMRI_REST1_RL_Atlas_MSMAll_hp2000_clean.dtseries.nii'):
        print('executing subject: '+dir+'......')
        
        img = nib.load(datadir_day1+dir+'/MNINonLinear/Results/rfMRI_REST1_RL/rfMRI_REST1_RL_Atlas_MSMAll_hp2000_clean.dtseries.nii')
        data = img.get_fdata()
        data_parc = hcp.parcellate(data, hcp.mmp)
        data_sub = data_parc[:,360:]  # this is the subcortical regions parcels: include if you want 419x419 matrices
        
        img_cortex = data[:,hcp.struct.cortex] # cortex vertices
        node = [None]*400 # data_sub.shape[0] is time points. not all data has 1200 ts
        
        for i in range(400):
            a = img_cortex[:,np.where(schaefer_400==i+1)][:,0]
            ts = [None] * data_sub.shape[0]
        
            for n in range(data_sub.shape[0]):
                ts[n] = a[n].mean()
                node[i] = ts
      
        # cortex parcels finished, now concatenate cortex and subcortex
        data_cor = np.array(node)
        
        # if you want matrices to include cortical and subcortical regions parcels, concatenate them here (so that in 419x419 matrices, the last 19 are subcortical parcels)
        #data_cor_sub = np.concatenate((data_cor,data_sub.T))  
        
        # correlation
        corr_matrix = np.corrcoef(data_cor)  # use data_cor_sub if I want to include subcortical areas to yield 419x419 matrices
        corr_matrix[corr_matrix>0.99999]=1  # if 0.99999 make it 1 - for some unknown reason some correlations of same parcels are not round 1 so forcing it into 1
        
        # save
        fname_corr_matrix = dataout + 'DAY1_RL/corr/'+ dir +'.csv'
        np.savetxt(fname_corr_matrix, corr_matrix, delimiter = ',')
        print('finished subject: '+dir+'......')
    
    else:
        fname_corr_matrix_failed = dataout + 'DAY1_RL/file_failed.txt'
        fail_file = open(fname_corr_matrix_failed, 'a')
        fail_file.write(dir+'\n')
        fail_file.close()
        print('subject '+dir+' file dose not exist.....')


executing subject: 100206......


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


finished subject: 100206......


## Average functional connectivity and z-transform them across the 4 sessions

In [6]:
# (across rfMRI_REST1_LR, rfMRI_REST1_RL, rfMRI_REST2_LR, rfMRI_REST2_RL)

for dir in path_list:
    if (os.path.exists(dataout + 'DAY1_LR/corr/' + dir + '.csv') and
        os.path.exists(dataout + 'DAY1_RL/corr/' + dir + '.csv') and
        os.path.exists(dataout + 'DAY2_LR/corr/' + dir + '.csv') and
        os.path.exists(dataout + 'DAY2_RL/corr/' + dir + '.csv')):
        
        print('executing subject: '+dir+'......')
        
        LR1 = np.array(pd.read_csv(dataout + 'DAY1_LR/corr/' + dir + '.csv', header=None))
        RL1 = np.array(pd.read_csv(dataout + 'DAY1_RL/corr/' + dir + '.csv', header=None))
        LR2 = np.array(pd.read_csv(dataout + 'DAY2_LR/corr/' + dir + '.csv', header=None))
        RL2 = np.array(pd.read_csv(dataout + 'DAY2_RL/corr/' + dir + '.csv', header=None))
        
        # average across four sessions
        mean = (LR1+RL1+LR2+RL2)/4
        
        # Fisher r to z transform
        fc = np.arctanh(mean) 
        fc[fc == inf] = 0
        np.savetxt(dataout + 'mean/' + dir + '.csv', fc, delimiter = ',')
  
    else:
        unmatch = open(dataout + 'unmatch_to_calculate_mean.txt', 'a')
        unmatch.write(dir+'\n')
        unmatch.close()
        print('subject: '+dir+'  file does not exist for all sessions.....')

subject: .datalad  file does not exist for all sessions.....
subject: .git  file does not exist for all sessions.....
subject: .gitattributes  file does not exist for all sessions.....
subject: .gitmodules  file does not exist for all sessions.....
subject: .hcp  file does not exist for all sessions.....
executing subject: 100206......


/tmp/ipykernel_9990/2709286811.py:20: RuntimeWarning: divide by zero encountered in arctanh
  fc = np.arctanh(mean)


executing subject: 100307......
executing subject: 100408......
executing subject: 100610......
executing subject: 101006......
executing subject: 101107......
executing subject: 101309......
subject: 101410  file does not exist for all sessions.....
executing subject: 101915......
executing subject: 102008......
executing subject: 102109......
executing subject: 102311......
executing subject: 102513......
executing subject: 102614......
executing subject: 102715......
executing subject: 102816......
executing subject: 103010......
executing subject: 103111......
executing subject: 103212......
executing subject: 103414......
executing subject: 103515......
executing subject: 103818......
executing subject: 104012......
executing subject: 104416......
executing subject: 104820......
executing subject: 105014......
executing subject: 105115......
executing subject: 105216......
executing subject: 105620......
executing subject: 105923......
executing subject: 106016......
executing sub